In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from models import FeedForwardModel

In [ ]:
# model params
loss_type = 'cross_entropy'
optimizer_type = 'adam'
model = FeedForwardModel(loss_type, optimizer_type)

In [ ]:
# ff parameters
data_shape = (1,3,32,32)
layer_types = ['conv', 'conv', 'fc', 'fc', 'fc']
output_channels = [6, 16, 120, 84, 10]
kernel_sizes = [5]
act_types = ['relu']
pool_types = ["prob", "prob", None]
pool_ksizes = [2]
conv_strides = [1]
dropout = [None]

model.ff_network(data_shape, layer_types, output_channels, kernel_sizes, conv_strides, act_types, 
                 pool_types, pool_ksizes, dropout)

In [ ]:
# control params 
# cnt_layer_ids = [0,1]
# cnt_layer_types = ['conv', 'fc']
# cnt_output_channels = [16, 256]
# cnt_kernel_sizes = [5]

# model.control_network(cnt_layer_ids, cnt_layer_types, cnt_output_channels, cnt_kernel_sizes)

# Note: still broken (02/16/19)
# can initialize, but cannot run

In [ ]:
# run
random_input = torch.rand(data_shape)
model(random_input).shape

In [ ]:
# train
epochs = 2
train_loader = None # torch.utils.data.DataLoader
#model.train_this_shit(epochs, trainloader) # requires a torch.utils.data.DataLoader